![](https://www.math.unipd.it/~marcuzzi/BannerStrumentifondamentali.png)

# Shared memory e stencil

Nelle applicazioni che abbiamo visto finora ogni thread procede singolarmente, indipendentemente dagli altri threads. Ma questo non è possibile in ogni applicazione: a volte i thread devono comunicare tra loro o condividere dati, e anzi a volte è conveniente introdurre comunicazioni. Le interazioni tra thread possono formare dei colli di bottiglia nel codice poichè i threads cercano di accedere agli stessi dati in global memory, perciò i dati vengono caricati nella *shared memory*, una memoria più piccola ma molto più veloce della global memory. Solo i thread di uno stesso blocco possono comunicare tra loro. I casi tipici in cui è vantaggioso usare la shared memory sono i seguenti: 
- quando ci sono dipendenze tra i dati di thread diversi (ma dello stesso blocco)
- per diminuire il numero di accessi alla memoria global, favorendo il riutilizzo dei dati 
- quando il pattern degli accessi alla memoria global è irregolare (uncoalesced memory access)

In questo notebook vedremo un esempio nel quale i dati di threads che appartengono allo stesso intorno contrubiscono ad una combinazione lineare lineare i cui coefficienti sono costanti. Questa operazione prende il nome di filtraggio e l'array dei coefficienti della compinazione lineare è noto come filtro o *stencil*. Il raggio di uno stencil è il numero di elementi coperto dallo stencil su entrambi i lati. 

# Esempio: il calcolo delle derivate su una griglia 1D

Consideriamo il metodo delle differenze finite per approssimare le derivate di una funzione. Scegliamo una discretizzazione $t_i$, $i=1,...,N$, di $N$ punti equispaziati su un generico intervallo $[a,b]$. L'approssimazione alle differenze finite in avanti (forward difference) per la derivata prima nel punto $t_i$ è la seguente:
$$ \frac{dx}{dt}(t_i) = \frac{x_{i+1}-x_i}{h},$$ 

dove $x_i = x(t_i)$ e $h = t_{i+1} - t_{i}$ è la distanza tra due punti consecutivi. Se applichiamo di nuovo le differenze finite ottenuamo un'approssimazione alle differenze finite centrate per la derivata seconda nel punto $t_i$: 

$$ \frac{d^2x}{dt^2}(t_i) = \frac{x_{i-1}-2x_i+x_{i+1}}{h^2}.$$ 

Questa formula è valida tu tutto l'intervallo tranne che per gli estremi, poichè non sono definiti entrambi i punti $t_{i-1},t_{i+1}$.
In questo caso lo stencil è l'array di coefficienti `[1,-2,1]` e il suo raggio è `RAD = 1`.

![](https://i.postimg.cc/SN1mSPfN/shared-rad.png)

L'esercizio che proponiamo è molto semplice: consiste nel calcolare la derivata seconda della funzione $x(t) = sin(t)$ utilizzando l'approssimazione alle differenze finite centrate scritta sopra. Scegliamo una discretizzazione $t_i$, $i=1,...,N$, sull'intervallo $[0,2\pi]$. La verifica del risultato è immediata in quanto $x''(t) = -sin(t)$, perciò si dovrà avere

$$x(t_i) + x''(t_i) \approx 0, \quad i=1,..,N.$$


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import math
from numba import cuda
from numba import float32
import numpy as np
from timeit import default_timer

In [2]:
N = 128
h = 2 * np.pi / N

t = np.linspace(0, 2*np.pi, N)
x = np.sin(t)
ddx = np.zeros((N,), dtype = np.float32)

In [3]:
@cuda.jit
def ddKernel(ddx, x, h):
    i = cuda.grid(1)
    
    if (i>0) and (i<x.shape[0] - 1):
        ddx[i] = (x[i - 1] - 2.0*x[i] + x[i + 1]) / (h*h)
    #endif
    return

In [4]:
TPB = 32 #numero di thread per blocco
BPG = math.ceil(N/TPB) #numero di blocchi nella griglia

ddKernel[BPG, TPB](ddx, x, h)

In [5]:
print(ddx+x)

[ 0.00000000e+00 -7.71619907e-04 -1.54134913e-03 -2.30730628e-03
 -3.06761445e-03 -3.82042755e-03 -4.56389183e-03 -5.29616242e-03
 -6.01550540e-03 -6.72009123e-03 -7.40825515e-03 -8.07831608e-03
 -8.72858076e-03 -9.35744421e-03 -9.96346794e-03 -1.05450790e-02
 -1.11008738e-02 -1.16295737e-02 -1.21297520e-02 -1.26002274e-02
 -1.30399311e-02 -1.34476670e-02 -1.38225389e-02 -1.41635555e-02
 -1.44699039e-02 -1.47408781e-02 -1.49757617e-02 -1.51740216e-02
 -1.53350989e-02 -1.54586904e-02 -1.55443885e-02 -1.55921111e-02
 -1.56017104e-02 -1.55730149e-02 -1.55063019e-02 -1.54015939e-02
 -1.52592201e-02 -1.50794992e-02 -1.48628537e-02 -1.46098788e-02
 -1.43211275e-02 -1.39973269e-02 -1.36392508e-02 -1.32478436e-02
 -1.28240006e-02 -1.23687773e-02 -1.18832546e-02 -1.13687130e-02
 -1.08262828e-02 -1.02574397e-02 -9.66343789e-03 -9.04576694e-03
 -8.40602214e-03 -7.74566472e-03 -7.06635682e-03 -6.36973812e-03
 -5.65756732e-03 -4.93154249e-03 -4.19342542e-03 -3.44508186e-03
 -2.68828795e-03 -1.92491

Valori uniformamente vicini allo zero ci confermano la validità, ma anche i limiti dell'accuratezza dell'approssimazione alle differenze finite.

Notiamo che il calcolo della derivata seconda comporta trasferimenti superflui di memoria dalla global memory: poichè ogni thread legge i dati in input associati al suo indice e a a quelli immediatamente vicini, ogni elemento dell'array in input viene letto 3 volte (`x[i]` viene letto come valore destro, come valore centrale e come valore sinistro rispettivamente dai thread con indice `i-1`, `i`, `i+1`).

Utilizziamo quindi la shared memory per effettuare un'unica lettura dei dati in input dalla global memory. L'idea è quella spezzare i dati input output in maniera tale da fornire ad ogni blocco di thread esattamente i dati di cui ha bisogno. Creiamo quindi un array in memoria shared dove andare salvare questi dati. Ogni thread avrà due indici
- l'indice globale `i` per accedere alla memoria global;
- l'indice locale `s_i` per accedere alla memoria shared.

Notiamo che non è sufficiente copiare solo i valori di x corrispondenti ai thread nel blocco; avremo bisogno anche dei valori che vengono coperti dallo stencil quando viene applicato agli estremi del blocco, perciò dovremmo includere tanti elementi quanto è il raggio dello stencil ad ogni estremo. Questi elementi aggiuntivi prendono il nome di *Halo*. Ogni blocco avrà bisogno di un array shared memory di lunghezza `TBP+2*RAD`, dove `TPB` è il numero di thread per blocco (`blockDim.x`).

![](https://i.postimg.cc/VkSmdFK9/shared-halo.png)

Il procedimento tipico consiste nei seguenti step
- dichiarare un array shared di dimensione `TPB + 2*RAD` (le due parti di lunghezza `RAD` prendono il nome di *halo* destro e sinistro)
- caricare `TPB + 2*RAD` dati in shared memory
- verificare che tutti i thread abbiamo terminato il caricamento attraverso una **barriera di sincronizzazione**
- calcolare `TPB` elementi in output

A volte anche il risultato viene calcolato il shared memory. In questo caso ènecessario un altro step
- copiare `TPB` elementi di output dalla shared memory di nuovo nella global memory



In [18]:
RAD = 1 #raggio
S_MEMSIZE = TPB + 2*RAD

@cuda.jit
def ddKernel_shared(ddx, x, h):
    i = cuda.grid(1)
    s_i  = cuda.threadIdx.x
    x_sh = cuda.shared.array(shape=(S_MEMSIZE), dtype = float32)
    
    if (i < ddx.shape[0]): 
        # dati i_1, ..., i_TPB
        x_sh[s_i + RAD] = x[i]
    
        # dati aggiuntivi i_1-1 e i_TPB+1
        if (cuda.threadIdx.x < RAD):
            x_sh[s_i] = x[i - RAD]
            x_sh[s_i + cuda.blockDim.x + RAD] = x[i + cuda.blockDim.x]
        #endif
        s_i = s_i + RAD
        cuda.syncthreads()
        if (i>0) and (i<x.shape[0] - 1):
            ddx[i] = (x_sh[s_i-1] - 2.0*x_sh[s_i] + x_sh[s_i+1])/(h*h)
        #endif
    #endif
    return

## Il codice nel dettaglio

- Dichiariamo un array in shared memory di dimensione `TPB + 2*RAD` 

       x_sh = cuda.shared.array(shape=(S_MEMSIZE), dtype = float32)
   
   ![](https://i.postimg.cc/Z51Z3yQG/shared1.png)

- Carichiamo in shared memory i dati centrali

        x_sh[s_i + RAD] = x[i]
        
     ![](https://i.postimg.cc/g08pK2Fn/shared2.png)
     
- Carichiamo l'halo sinistro (solo thread con indici `0,...,RAD-1`)
 
         x_sh[s_i] = x[i - RAD]
         
     ![](https://i.postimg.cc/wvzp4YgF/shared3.png)
     
- Carichiamo l'halo destro (solo thread con indici `0,...,RAD-1`)
 
         x_sh[RAD + s_i + cuda.blockDim.x] = x[i + cuda.blockDim.x]
         
     ![](https://i.postimg.cc/pTBtgcdv/shared4.png)
     
- Barriera di sincronizzazione, per assicurarsi che tutti i thread abbiano completato il caricamento in shared

        cuda.syncthreads()
        
- Calcolo del risultato (in questo caso in global)

        ddx[i] = (x_sh[s_i-1] - 2.0*x_sh[s_i] + x_sh[s_i+1])/(h*h)

In [19]:
ddKernel_shared[BPG, TPB](ddx, x, h)

In [20]:
print(ddx+x)

[ 0.00000000e+00 -7.70889750e-04 -1.54529793e-03 -2.30171835e-03
 -3.07372393e-03 -3.81664265e-03 -4.56508392e-03 -5.29342061e-03
 -6.01756176e-03 -6.72044886e-03 -7.41364937e-03 -8.05667959e-03
 -8.75462799e-03 -9.33735745e-03 -9.97771345e-03 -1.05481784e-02
 -1.10904430e-02 -1.16174739e-02 -1.21627134e-02 -1.25820480e-02
 -1.30233610e-02 -1.34814628e-02 -1.37977433e-02 -1.41792911e-02
 -1.44570293e-02 -1.47532759e-02 -1.49619930e-02 -1.51695512e-02
 -1.53575699e-02 -1.54523723e-02 -1.55228117e-02 -1.56070123e-02
 -1.56135122e-02 -1.55687234e-02 -1.54927121e-02 -1.53998058e-02
 -1.52735252e-02 -1.50669226e-02 -1.48753707e-02 -1.46155413e-02
 -1.42971068e-02 -1.40232549e-02 -1.36195216e-02 -1.32523735e-02
 -1.28356831e-02 -1.23524456e-02 -1.18797379e-02 -1.13907667e-02
 -1.08049443e-02 -1.02844406e-02 -9.64120536e-03 -9.05411159e-03
 -8.38921363e-03 -7.76879132e-03 -7.05217092e-03 -6.36780097e-03
 -5.66823655e-03 -4.92841324e-03 -4.19521356e-03 -3.43946412e-03
 -2.68912241e-03 -1.92750

In [25]:
t_i = default_timer()
ddKernel[BPG, TPB](ddx, x, h)
t_f = default_timer()
ddKernel_time = t_f - t_i

t_i = default_timer()
ddKernel_shared[BPG, TPB](ddx, x, h)
t_f = default_timer()
cuda.synchronize()
ddKernel_sh_time = t_f - t_i

print('Tempi di calcolo senza shared = ', ddKernel_time)
print('Tempi di calcolo con shared = ', ddKernel_sh_time)

Tempi di calcolo senza shared =  0.0026648957282304764
Tempi di calcolo con shared =  0.001457851380109787


Questo esempio è preso da: 
*CUDA for Engineers: An Introduction to High-Performance Parallel Computing*, Mete Yurtoglu, Duane Storti.